In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

#list_of_filters = ["maternity-wear-dress","maternity-wear-top","modest","overalls","overcoat","polo-t-shirt","pullover","raincoat","sets"]
#list_of_filters = ["shawl","shirt","short","skirt","sportswear","sweatshirt","swimming-bottoms"]
#list_of_filters =["swimming-shorts","swimming-tops","swimsuit","t-shirt","tights","trenchcoat","trenchcoat-raincoat","trousers","underwear","vest","beach-wear"]
#list_of_filters =["tunics","athlete","beachwear","bermuda","bikini","blazer-jacket","blouse","bra","burkini","capri"]
list_of_filters =["cardigans","coat","defacto-fit","dress","homewear","homewear-bottom","homewear-top","jacket","jeans","jumpsuit","leather-jacket"
,"leggings","lingerie-bottoms","lingerie-sets","lingerie-tops","maternity-wear","maternity-wear-bottom","maternity-wear-denim"]


# Setup Selenium ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Base URL of the webpage to scrape
base_url = "https://www.defacto.com/en-iq/woman-"

# List to store all the DataFrames
dfs = []


for filter_item in list_of_filters:
    # Construct the full URL with the current filter item
    url = f"{base_url}{filter_item}"
    
    # Open the webpage
    driver.get(url)
    
    # Wait for the initial content to load
    wait = WebDriverWait(driver, 10)
    
    # List to store product details
    products_list = []
    
    # Repeatedly click the "Load More Products" button
    while True:
        try:
            # Check if all products are loaded
            load_more_info = driver.find_element(By.CLASS_NAME, 'load-more__progress-content')
            data_loaded = int(load_more_info.get_attribute('data-loaded'))
            data_all = int(load_more_info.get_attribute('data-all'))
            if data_loaded >= data_all:
                print(f"All products loaded: {data_loaded} / {data_all}")
                break
                
            # Check if the Load More button is visible and clickable
            load_more_button = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'load-more__button')))
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            load_more_button.click()
            # Wait for the products to load
            time.sleep(3)  # Adjust this based on your network speed
            
            # Check if all products are loaded
            load_more_info = driver.find_element(By.CLASS_NAME, 'load-more__progress-content')
            data_loaded = int(load_more_info.get_attribute('data-loaded'))
            data_all = int(load_more_info.get_attribute('data-all'))
            if data_loaded >= data_all:
                print(f"All products loaded: {data_loaded} / {data_all}")
                break
                
        except Exception as e:
            # If Load More button is not found or not clickable, break the loop
            print("An error occurred:", str(e))
    
    # Scroll to the bottom of the page to load all content
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Additional wait for any lazy-loaded images
    time.sleep(3)
    
    # Find all product containers
    product_containers = driver.find_elements(By.CLASS_NAME, 'catalog-products__item')
        
    # Iterate through each product container and extract details
    for product in product_containers:
        # Scroll to each product element using JavaScript (helpful for lazy loading)
        driver.execute_script("arguments[0].scrollIntoView(true);", product)
        time.sleep(1)  # Small delay to allow lazy-loaded images to load
    
        # Extract product name
        name_elements = product.find_elements(By.CSS_SELECTOR, '.product-card__title')
        name = name_elements[0].text if name_elements else 'No Name'
        
        # Extract product price
        price_elements = product.find_elements(By.CSS_SELECTOR, '.product-card__price--new')
        price = price_elements[0].text if price_elements else 'No Price'
        
        # Extract image URL with retry mechanism
        retry_count = 0
        max_retries = 3
        image_url = None
        while retry_count < max_retries and not image_url:
            image_elements = product.find_elements(By.CSS_SELECTOR, 'img.swiper-lazy')
            if image_elements:
                image_url = image_elements[0].get_attribute('srcset').split(",")[0].split(" ")[0]
            time.sleep(1)  # Short wait before retry
            retry_count += 1
    
        first_image_url = image_url if image_url else 'No Image URL'
    
        # Extract the original price
        original_price_elements = product.find_elements(By.CSS_SELECTOR, ".product-card__price--new")
        original_price = original_price_elements[0].text if original_price_elements else 'No Original Price'
        
        # Extract the discounted price
        discounted_price_elements = product.find_elements(By.CSS_SELECTOR, ".product-card__price--basket .sale")
        discounted_price = discounted_price_elements[0].text if discounted_price_elements else 'No Discounted Price'
        
        # Extract the discount percentage
        discount_percentage_elements = product.find_elements(By.CSS_SELECTOR, ".product-card__price--basket .percent")
        discount_percentage = discount_percentage_elements[0].text if discount_percentage_elements else 'No Discount Percentage'

        product_details = {
            'Product Category': filter_item,
            'Product Name': name,
            'Price': price,
            'Image URL': first_image_url,
            'Original Price': original_price,
            'Discounted Price': discounted_price,
            'Discount': discount_percentage
        }
        products_list.append(product_details)
    # Create a DataFrame for the current category and add it to the list
    df = pd.DataFrame(products_list)
    dfs.append(df)
    
# Concatenate all the DataFrames
final_df = pd.concat(dfs, ignore_index=True)

# Save the DataFrame to an Excel file
final_df.to_excel('Defacto 2nd trial.xlsx', index=False)

# Close the browser
driver.quit()


An error occurred: Message: element click intercepted: Element is not clickable at point (589, 11847)
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x005E6EE3+174339]
	(No symbol) [0x00510A51]
	(No symbol) [0x00226FF6]
	(No symbol) [0x0025E48E]
	(No symbol) [0x0025D09E]
	(No symbol) [0x0025B5F8]
	(No symbol) [0x0025AD7F]
	(No symbol) [0x00252B4E]
	(No symbol) [0x0027700C]
	(No symbol) [0x002525B0]
	(No symbol) [0x00277414]
	(No symbol) [0x0028A104]
	(No symbol) [0x00276DA6]
	(No symbol) [0x00251034]
	(No symbol) [0x00251F8D]
	GetHandleVerifier [0x00684B1C+820540]
	sqlite3_dbdata_init [0x007453EE+653550]
	sqlite3_dbdata_init [0x00744E09+652041]
	sqlite3_dbdata_init [0x007397CC+605388]
	sqlite3_dbdata_init [0x00745D9B+656027]
	(No symbol) [0x0051FE6C]
	(No symbol) [0x005183B8]
	(No symbol) [0x005184DD]
	(No symbol) [0x00505818]
	BaseThreadInitThunk [0x76687BA9+25]
	RtlInitializeExceptionChain [0x77DEBD2B+107]
	RtlClearBits [0x77DEBCAF+191]

An error occurred: Me